In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os
import pandas as pd
import torch.nn as nn
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split, Dataset
 
from tqdm import tqdm
from utils.process_data import build_train_test_loaders, build_train_test_loaders_2
from utils.training import CustomMAELoss, CustomMAPELoss, test_model, get_flow_forecasting_metrics
from utils.link_loads import get_graph_attributes, df_to_graph, build_quarter_hour_data, add_missing_nodes, add_missing_nodes_2
from utils.models import STGCN
# import networkx as nx

- Charger les csv
- Supprimer les horaires 00h - 05h
- Les save by jour
- Créer la class Dataset qui renvoie le graph actuel avec les flows précédent dans la journée.

In [3]:
folder_path = "data/"
# Get graph attributes, create dfs from csv, process Link column and ordered dfs by time
num_nodes, edge_index, node_mapping, dfs = get_graph_attributes(folder_path)

# Training

In [4]:
MAE = CustomMAELoss()
MAPE = CustomMAPELoss()

### GCN + GRU (pas concluant, ne pas run)

In [5]:
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.extend(df_qhrs)
    
graphs = [df_to_graph(df, edge_index) for df in graph_data]  # Un graphe par quart d'heure

In [6]:
class TGCN(nn.Module):
    def __init__(self, node_features, hidden_dim, gru_hidden_dim):
        super(TGCN, self).__init__()
        self.gcn = GCNConv(node_features, hidden_dim)  # GCN
        self.gru = nn.GRU(hidden_dim, gru_hidden_dim, batch_first=True)  # GRU
        self.fc = nn.Linear(gru_hidden_dim, 1)  # Prédiction finale

    def forward(self, graph_seq):
        # window_size = len(graph_seq)  # Nombre de pas de temps
        # batch_size = graph_seq[0].x.shape[0]  # Nombre de nœuds

        spatial_features = []
        for graph in graph_seq:
            x = self.gcn(graph.x, graph.edge_index)  # GCN
            x = F.relu(x)
            spatial_features.append(x)

        spatial_features = torch.stack(spatial_features, dim=1)  # (batch, time, hidden_dim)

        _ , final_state = self.gru(spatial_features)  # prédiction sur le dernier (1, gru_hidden_dim)
        final_state = final_state.squeeze() # (gru_hidden_dim)
        final_out = self.fc(final_state) # Prédiction sur le dernier état
        final_out = F.relu(final_out)
        return final_out

In [ ]:
graphs[0]

In [ ]:
model = TGCN(node_features=10, hidden_dim=32, gru_hidden_dim=64)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = MAE

# Boucle d'entraînement
for epoch in tqdm(range(50)):
    model.train()
    total_loss = 0
    for graph_seq, target in train_loader:
        optimizer.zero_grad()
        output = model(graph_seq)

        target = target.squeeze()
        output = output.reshape(target.shape)
    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")


In [ ]:
model.eval()
test_loss = 0
with torch.no_grad():
    for graph_seq, target in test_loader:

        output = model(graph_seq)
        loss = criterion(output, target)
        test_loss += loss.item()

print(f"Test MAE: {test_loss / len(test_loader)}")


In [ ]:
39e6/(96*1206)

The average flow per inter_station per 15min is 337. So 204 MAE error is not a valid result.

### GCN with past flows as node features

In [7]:
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.extend(df_qhrs)
    
graphs = [df_to_graph(df, edge_index) for df in graph_data]  # Un graphe par quart d'heure

window_size=4

train_loader, test_loader = build_train_test_loaders(graphs, window_size)

In [8]:
loss_mae = 0
for graph, temporal_features in test_loader:
    output = graph.x[:,-1]
    output = output.unsqueeze(-1)
    target = graph.y

    loss_mae += MAE(output, target)

print("En utilisant la dernière valeur du flow comme prédiction :")
print(f"Test MAE: {loss_mae / len(test_loader)}")

En utilisant la dernière valeur du flow comme prédiction :
Test MAE: 21.719919204711914


In [ ]:
def is_nan_or_empty(tensor):
    return tensor.numel() == 0 or torch.isnan(tensor).all()

loss_mae = 0
loss_mape = 0
for graph, temporal_features in test_loader:
    output = graph.x[:,-1]
    output = output.unsqueeze(-1)
    target = graph.y

    maskP = target >= 1
    countP = maskP.sum().item()

    outputP = output[maskP]
    targetP = target[maskP]

    if not (is_nan_or_empty(outputP) or is_nan_or_empty(targetP)):
        loss_mae += MAE(outputP, targetP)
        loss_mape += MAPE(outputP, targetP)

print("En utilisant la dernière valeur du flow comme prédiction :")
print(f"Test MAE: {loss_mae / len(test_loader)}")
print(f"Test MAPE: {loss_mape / len(test_loader)}")

MAPE pas de sens ici à cause des valeurs nulles

Compliqué d'utiliser la MAPE car trop de targets égale ou proche de zéro

### GCN 0500-0000 flows

In [10]:
window_size = 4

In [11]:
# Build the Data loader
graph_data = []
# each df should have the same dimension and same nodes at the same columns
for filename, df in dfs.items():
    df = add_missing_nodes_2(df, node_mapping, num_nodes) # add zeros row for missing nodes
    df_qhrs = build_quarter_hour_data(df, filename, num_nodes) # retourne 24*4 df avec ses paramètres temporel et le flow
    graph_data.append(df_qhrs)
graphs = [[df_to_graph(df, edge_index) for df in graph_data[i]] for i in range(len(graph_data))]  # Un graphe par quart d'heure

train_loader, test_loader = build_train_test_loaders_2(graphs, window_size, train_split=0.8)

In [12]:
# Performance de la Baseline (prédiction du dernier flow)
loss_mae = 0
loss_mape = 0
count_total = 0
MAPE_high_target = 0
MAE_low_target = 0

for graph, temporal_features in test_loader:
    output = graph.x[:,-1]
    output = output.unsqueeze(-1)
    target = graph.y
    loss_mae += MAE(output, target)
    loss_mape += MAPE(output, target)
    MAPE_high_target_ , MAE_low_target_ = get_flow_forecasting_metrics(output, target)
    MAPE_high_target += float(MAPE_high_target_)
    MAE_low_target += float(MAE_low_target_)

print("En utilisant la dernière valeur du flow comme prédiction :")
print(f"Test MAE: {loss_mae / len(test_loader)}")

print(f"MAPE for high targets: {MAPE_high_target / len(test_loader)}")
print(f"MAE for low targets: {MAE_low_target/ len(test_loader)}")

En utilisant la dernière valeur du flow comme prédiction :
Test MAE: 25.606801986694336
MAPE for high targets: 0.07953206537705329
MAE for low targets: 8.892926945572807


L'objectif étant de faire mieux que cette base

Ci dessous voici les performances de la baseline sans prendre en compte les valeurs nulles 

In [ ]:
def is_nan_or_empty(tensor):
    return tensor.numel() == 0 or torch.isnan(tensor).all()

loss_mae = 0
loss_mape = 0
for graph, temporal_features in test_loader:
    output = graph.x[:,-1]
    output = output.unsqueeze(-1)
    target = graph.y

    maskP = target >= 1
    countP = maskP.sum().item()

    outputP = output[maskP]
    targetP = target[maskP]

    if not (is_nan_or_empty(outputP) or is_nan_or_empty(targetP)):
        loss_mae += MAE(outputP, targetP)
        loss_mape += MAPE(outputP, targetP)

print("En utilisant la dernière valeur du flow comme prédiction :")
print(f"Test MAE: {loss_mae / len(test_loader)}")
print(f"Test MAPE: {loss_mape / len(test_loader)}")

Training:

In [13]:
model = STGCN(node_features=window_size, hidden_dim=32)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 40

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()        
        train_loss += loss.item()

    scheduler.step()
    
    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1, Train loss: 33.027031764586766


  2%|▎         | 1/40 [00:13<08:33, 13.18s/it]

Epoch 1, Test loss: 23.617153944477202
Epoch 2, Train loss: 19.945948464738


  5%|▌         | 2/40 [00:25<08:04, 12.76s/it]

Epoch 2, Test loss: 24.19250312967906


  5%|▌         | 2/40 [00:26<08:27, 13.36s/it]


KeyboardInterrupt: 

In [ ]:
test_model(model, test_loader)

### Improve the last model

In [ ]:
from utils.models import ImprovedSTGCN
model = ImprovedSTGCN(node_features=window_size, hidden_dim=16)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = MAE

num_epochs = 40

# Boucle d'entraînement
for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0
    for graph, temporal_features in train_loader:
        temporal_features = temporal_features.squeeze()
        optimizer.zero_grad()
        output = model(graph, temporal_features)
        target = graph.y    
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()        
        train_loss += loss.item()

    scheduler.step()
    
    print(f"Epoch {epoch+1}, Train loss: {train_loss / len(train_loader)}")

    # Évaluation sur le jeu de test
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Désactivation des gradients pour l'évaluation
        for graph, temporal_features in test_loader:
            temporal_features = temporal_features.squeeze()  # Squeeze si nécessaire
            output = model(graph, temporal_features)  # Assure-toi de passer aussi les temporal_features
            target = graph.y
            loss = criterion(output, target)
            test_loss += loss.item()

    print(f"Epoch {epoch+1}, Test loss: {test_loss / len(test_loader)}")

### Some EDA

In [ ]:
mean = 0
for graph, temporal_features in test_loader:
    y = graph.y
    mean += y.mean()
print(f"Average of passengers per inter-stations (nodes): {mean/len(test_loader)}")

In [ ]:
import torch
import matplotlib.pyplot as plt

# Définition des quantiles à calculer
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
quantile_values = {q: 0 for q in quantiles}

# Calcul des quantiles sur l'ensemble du test_loader
for graph, temporal_features in test_loader:
    y = graph.y
    for q in quantiles:
        quantile_values[q] += y.quantile(q)

# Moyenne des quantiles sur l'ensemble du dataset
num_graphs = len(test_loader)
quantile_averages = {q: quantile_values[q] / num_graphs for q in quantiles}

# Tracé du graphique
plt.figure(figsize=(8, 5))
plt.plot(quantiles, list(quantile_averages.values()), marker='o', linestyle='-', color='b')
plt.xlabel("Quantiles")
plt.ylabel("Valeur moyenne du flux")
plt.title("Quantiles moyens des flux de passagers par inter-station")
plt.grid(True)
plt.show()

In [ ]:
quantile_averages